# __Please run the provided demo notebook file in Google Colab to explore the hands-on example.__

# **Run LLM Falcon Locally**

# **Description:**
This project will walk you through the process of setting up a local Falcon LLM using Langchain’s prompt template and conversationChain functionalities.

# **Steps to Perform:**
1. Set up the Environment
2. Download Falcon 7B Model and Tokenizer from Hugging Face
3. Set up Model and Generation Configuration
3. Build the Conversation Chain
4. Modify the Prompt Template to Define a Specific Conversational Style
5. Manage Conversation History with Conversationbufferwindowmemory
6. Interact with the LLM


# **Step 1: Set up the Environment**

In [ ]:
#Install the libraries if not installed
#!pip install bitsandbytes, langchain, torch, transformers, accelerate, xformers, einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

# **Step 2: Download the Falcon 7B Model and Tokenizer from Hugging Face**

In [ ]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_8bit=True, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Model device: {model.device}")


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Model device: cuda:0


# **Step 3: Set up the LLM and Generation Configuration**
Configure the LLM for inference using the following Python code:

In [ ]:

model.eval()
generation_config = model.generation_config
# Set temperature to 0 for deterministic responses
generation_config.temperature = 0
# Set number of returned sequences to 1
generation_config.num_return_sequences = 1
# Set maximum new tokens per response
generation_config.max_new_tokens = 256
# Disable token caching
generation_config.use_cache = False
# Set repetition penalty for more diverse responses
generation_config.repetition_penalty = 1.7
# Define pad and EOS token IDs
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# **Step 4: Build the Conversation Chain**


*   Define a custom prompt template that sets the context for the conversation.
*   Create the ConversationChain object using the following Python code:





In [ ]:

initial_prompt = """
The following is a conversation between a human and an AI. The AI is knowledgeable and provides detailed answers.

Current conversation:

Human: What is the theory of relativity?
AI:
""".strip()

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)

# Create the HuggingFacePipeline object
llm_pipeline = HuggingFacePipeline(pipeline=pipe)

# Create the ConversationChain object
dialogue_chain = ConversationChain(llm=llm_pipeline)

# Print the initial prompt template
print(dialogue_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


# **Step 5: Modify the Prompt Template to Define a Specific Conversational Style**

In [ ]:
new_template = """
The following is a conversation between a human and an AI. The AI behaves like Albert Einstein, providing detailed explanations about physics.

Current conversation:
{history}
Human: {input}
AI:""".strip()

# Create a new PromptTemplate object
prompt = PromptTemplate(input_variables=["history", "input"], template=new_template)

# Print the new prompt template
print(new_template)


The following is a conversation between a human and an AI. The AI behaves like Albert Einstein, providing detailed explanations about physics.

Current conversation:
{history}
Human: {input}
AI:


# **Step 6: Manage Conversation History with Conversationbufferwindowmemory**

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

# **Step 7: Interact with the LLM**
* Provide an input prompt to initiate the conversation and start interacting with the LLM.
* Observe the chain’s output and continue the dialogue by providing further input.

In [ ]:
text = "Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce."
res = chain.predict(input=text)
print(res)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and an AI. The AI behaves like Albert Einstein, providing detailed explanations about physics.

Current conversation:

Human: Think of a name for automaker that builds family cars with big V8 engines. The name must be a single word and easy to pronounce.
AI:


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



> Finished chain.
 Tesla.
User 


# **Conclusion**
This tutorial provides a basic understanding of running a local Falcon LLM using Langchain’s PromptTemplate and ConversationChain functionalities. \
